Part of this notebook is taken from the tutorial-notebooks: 07b and 08 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import warnings
from lsst.rsp import get_tap_service, retrieve_query
import gc

# Astropy
import astropy.units as u
from astropy.units import UnitsWarning
from astropy.timeseries import LombScargle

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

In [2]:
%matplotlib inline

In [3]:
params = {'axes.labelsize': 24,
          'font.size': 20,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [10, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [4]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

In [5]:
#Initializing TAP
service = get_tap_service()

## Plotting functions

In [11]:
# Plot 
def plotlc(days, magnitudes, band, out_name):
    fig = plt.figure(figsize=(8,8))
    plt.plot(days[band], magnitudes[band],
             'k.', ms=10)
    plt.minorticks_on()
    plt.xlabel('MJD (days)')
    plt.ylabel(band)
    plt.savefig(out_name)
    
    return fig
    #plt.show() 
    
def make_figure(exp, out_name):
    """
    Create an image.
    should be followed with remove_figure

    Parameters
    ----------
    exp : calexp from butler.get
    out_name : file name where you'd like to save it
    
    """
    fig = plt.figure(figsize=(10, 8))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(exp.image)
    plt.gca().axis('on')
    plt.savefig(out_name)
    
    return fig
    
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure

    gc.collect()    # call the garbage collector
    

## Creating directories to save lightcurves and coadds

In [12]:
plots = []
bands = ['g','r','i']
batch_dir = "./lightcurves/"

if os.path.isdir(batch_dir+'/lc_plots') == False:
    os.mkdir(batch_dir+'/lc_plots')
else:
    os.system('rm -r '+batch_dir+'/lc_plots/*')
    
if os.path.isdir(batch_dir+'/text_files') == False:
    os.mkdir(batch_dir+'/text_files')
else:
    os.system('rm -r '+batch_dir+'/text_files/*')
    
if os.path.isdir(batch_dir+'/coadd_iamges') == False:
    os.mkdir(batch_dir+'/coadd_images')
else:
    os.system('rm -r '+batch_dir+'/coadd_images/*')

## Query to get variable stars

Selecting stars (truth_type=2)

variable (is_variable = 1)

is_pointsource = 1

In [6]:
def query_stars(ra_deg, dec_deg, radius_deg, limit):
    #radius within which you want to query
    
    query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec "\
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
            "WHERE ts.truth_type=2 "\
            "AND ts.is_variable = 1 "\
            "AND ts.is_pointsource = 1 "\
            "AND mt.match_objectId > 1 "\
            "AND CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', "+ str(ra_deg)+", "+str(dec_deg)+", "+str(radius_deg)+")) <= 1 "\
            "LIMIT "+str(limit)+" "
    results = service.search(query)
    variable_stars = results.to_table().to_pandas()
    return variable_stars

In [7]:
%%time
variable_stars = query_stars(62.1, -35.8, 0.10,10)

CPU times: user 18.6 ms, sys: 81 µs, total: 18.7 ms
Wall time: 4min 7s


In [8]:
variable_stars

,id_truth_type,match_objectId,ra,dec
0,31303792873_2,1567120728929628330,57.629630,-37.367242
1,30830360667_2,1567111932836619124,57.745865,-37.240226
2,31412100519_2,1567111932836619129,57.740697,-37.239736
3,31412103576_2,1567111932836617040,57.896241,-37.260708
4,31412103311_2,1567111932836617041,57.895600,-37.259191
5,31107892157_2,1567111932836619208,57.920275,-37.238498
6,31412124559_2,1567120728929600169,57.547503,-37.369222
7,31412124801_2,1567120728929600085,57.567439,-37.371151
8,31500868957_2,1567111932836619172,57.718898,-37.238788
9,31412100500_2,1567111932836619171,57.719313,-37.239139


## Query to get the flux at all the bands

In [9]:
def query_flux(objid):
    query = "SELECT src.band, src.ccdVisitId, src.coord_ra, src.coord_dec, "\
            "src.objectId, src.psfFlux, src.psfFluxErr, "\
            "scisql_nanojanskyToAbMag(psfFlux) as psfMag, "\
            "visinfo.band, "\
            "visinfo.expMidptMJD "\
            "FROM dp02_dc2_catalogs.ForcedSource as src "\
            "JOIN dp02_dc2_catalogs.CcdVisit as visinfo "\
            "ON visinfo.ccdVisitId = src.ccdVisitId "\
            "WHERE src.objectId = "+str(objid)+" "
    lc = service.search(query)
    flux_allbands = lc.to_table()
    # Create masks for the bands
    pick = {}
    for filter in plot_filter_labels:
        pick[filter] = (flux_allbands['band'] == filter)
    mjd_days = {}
    mags = {}
    for filter in plot_filter_labels:
        mjd_days[filter] = np.array(flux_allbands[pick[filter]]['expMidptMJD']) * u.day
        mags[filter] = np.array(flux_allbands[pick[filter]]['psfMag'])
        
    return mjd_days, mags

## Query coadd images from Butler

In [17]:
from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

In [18]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get('skyMap')

In [19]:
for index, row in variable_stars.iterrows():
    deepCoadd= butler.get('deepCoadd', dataId=row['match_objectId'],band='i')
    figout = make_figure(deepCoadd, batch_dir+"/coadd_images/"+str(row['objectId'])+".png")
    cutouts.append(figout)
    remove_figure(figout)

AttributeError: 'int' object has no attribute 'items'

## Plotting images and text files of the light curves

In [49]:
%%time
for index, row in variable_stars.iterrows():
    df_all_bands = []
    mjd_days, mags = query_flux(row['match_objectId'])
    for band in bands:
        figout = plotlc(mjd_days, mags, band, batch_dir+"/lc_plots/"+"lc_"+str(row['match_objectId'])+'_'+band+".png")
        plots.append(figout)
        remove_figure(figout) 
        
        df = pd.DataFrame(data = {'band': [band]*len(test_mjd_days[band]), 'mjd_days': test_mjd_days[band], \
                          'mags': test_mags[band]}, index=None)
        df_all_bands.append(df)
    
    df_final = pd.concat(df_all_bands)
    outfile = batch_dir+"/text_files/"+"lc_"+str(row['match_objectId'])+".csv"
    df_final.to_csv(outfile, index=False, sep=',')

CPU times: user 11.7 s, sys: 2.28 s, total: 13.9 s
Wall time: 4min 19s


## Include Preprocessing to create phased light curves